# 合表

## 注意

1. `rpy2`来调用R代码，需要设`R_HOME`，否则会崩溃。
2. `%load_ext rpy2.ipython`之后，可以有`%%R`开头的纯R单元格，可以用`-i`传数据给R，或者`-o`提取数据，详情问ChatGPT。

In [7]:
import pandas as pd
import os
import sys
import numpy as np
sys.path.insert(0, '..')  # 将上一层目录添加到系统路径


from src.utils import *

reg_df = pd.concat([pd.read_csv('data/' + x) for x in os.listdir('data/')]).reset_index(drop=True)
reg_df['child_enroll'] = reg_df['child_enroll'].apply(lambda x:np.nan if x >1 else x)

In [8]:
desc(reg_df,drop=['pid','_wav','eth','enroll2','child_birth_prov'])

,count,min,max,mean,mode
wav,10302,2014.0,2020.0,2016.88,2014.0
child_birth_y,7705,1999.0,2010.0,2004.65,2004.0
child_age,10302,10.0,16.0,12.43,10.0
child_gender,10302,0.0,1.0,0.54,1.0
child_enroll,10182,0.0,1.0,0.96,1.0
child_math,10022,1.0,4.0,2.33,3.0
child_eduy,10236,0.0,13.0,6.00,6.0
mother_birth_y,6289,1938.0,1994.0,1977.47,1976.0
mother_age,9187,21.0,76.0,39.26,37.0
mother_gender,9190,0.0,1.0,0.00,0.0


In [9]:
## 合并出生省份

import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import sys

sys.path.insert(0, '..')  # 将上一层目录添加到系统路径

from src.reader import read_cfps, find_files, find_cfps

data_path = os.getenv('data_path')



In [10]:
find_files(data_path + '\\cfps',r'.*adult.*\.sas')

['C:\\Users\\lee\\nutcloud\\Data\\cfps\\2010\\cfps2010adult_202008.sas7bdat',
 'C:\\Users\\lee\\nutcloud\\Data\\cfps\\2010\\cfps2010adult_202008.sas7bdat.feather']

In [11]:
df = read_cfps('C:\\Users\\lee\\nutcloud\\Data\\cfps\\2010\\cfps2010adult_202008.sas7bdat')
df.head()

C:\Users\lee\nutcloud\Data\cfps\2010\cfps2010adult_202008.sas7bdat
feather存在，读取feather。


,pid,fid,cid,provcd,countyid,PSU,Rswt_Nat,Rswt_Res,subpopulation,subsample,...,foccupcode,foccupisco,FPARTY,MBIRTHY,moccupcode,moccupisco,MPARTY,Interviewerid,ReleaseVersion,wav
0,110001101.0,110001.0,11830.0,11.0,45.0,45.0,41928.792969,31068.037109,6.0,1.0,...,NaN,NaN,NaN,1938.0,NaN,NaN,NaN,11518.0,5.2,2010
1,110001102.0,110001.0,11830.0,11.0,45.0,45.0,37480.898438,25093.058594,6.0,1.0,...,NaN,NaN,NaN,1934.0,NaN,NaN,NaN,11518.0,5.2,2010
2,110003101.0,110003.0,11830.0,11.0,45.0,45.0,43325.535156,32052.169922,6.0,1.0,...,20603.0,2411.0,4.0,1943.0,80000.0,NaN,4.0,11518.0,5.2,2010
3,110003102.0,110003.0,11830.0,11.0,45.0,45.0,38129.796875,27176.064453,6.0,1.0,...,20904.0,2331.0,1.0,1948.0,80000.0,NaN,4.0,11518.0,5.2,2010
4,110005101.0,110005.0,11830.0,11.0,45.0,45.0,69329.976562,46488.992188,6.0,1.0,...,40301.0,5122.0,4.0,1965.0,NaN,NaN,4.0,11518.0,5.2,2010


In [12]:
birth_prov = df[['pid','QA102ACODE']]
birth_prov.columns = ['mother_pid','mother_birth_prov']

birth_prov.loc[birth_prov['mother_birth_prov'] < 0,'mother_birth_prov'] = np.nan
birth_prov.head()

C:\Users\lee\AppData\Local\Temp\ipykernel_59496\3316188949.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  birth_prov.loc[birth_prov['mother_birth_prov'] < 0,'mother_birth_prov'] = np.nan


,mother_pid,mother_birth_prov
0,110001101.0,21.0
1,110001102.0,21.0
2,110003101.0,32.0
3,110003102.0,61.0
4,110005101.0,11.0


In [13]:
from src.utils import safe_merge

reg_df = safe_merge(reg_df.drop('mother_birth_prov',axis=1),birth_prov)

Merge的方式是how='left'
注意：合并后的 DataFrame 有 5741/10302 个重复键值。第一个重复键值为: mother_pid    140394105.0
Name: 163, dtype: float64


In [14]:
desc(reg_df,drop=['pid','_wav','eth','enroll2','child_birth_prov'])

,count,min,max,mean,mode
wav,10302,2014.0,2020.0,2016.88,2014.0
child_birth_y,7705,1999.0,2010.0,2004.65,2004.0
child_age,10302,10.0,16.0,12.43,10.0
child_gender,10302,0.0,1.0,0.54,1.0
child_enroll,10182,0.0,1.0,0.96,1.0
child_math,10022,1.0,4.0,2.33,3.0
child_eduy,10236,0.0,13.0,6.00,6.0
mother_birth_y,6289,1938.0,1994.0,1977.47,1976.0
mother_age,9187,21.0,76.0,39.26,37.0
mother_gender,9190,0.0,1.0,0.00,0.0


## 回归


$$
Y_{it} = \beta_0 + \beta_1 S^{M}_{ipkt} + \beta_2 X_{it} + \theta^{M}_k + \tau^{M}_p + \omega_t + u_{it}
$$

In [15]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [16]:
%%R -i reg_df
ls()

[1] "check"             "lm0"               "lm1"              
[4] "reg_df"            "screenreg_unicode"


In [18]:
%%R
library(fixest)
source('../src/screenreg_unicode.R')

lm0 <- feols(child_math ~ mother_eduy, data = reg_df)
lm1 <- feols(child_math ~ mother_eduy + child_age + child_gender | wav + mother_birth_y + mother_birth_prov, data = reg_df)


check(list(lm0,lm1))
# 数学成绩1是优，4是差，因此母亲的教育对子女的数学成绩有显著的正影响


                                Model 1      Model 2    
--------------------------------------------------------
(Intercept)                        2.66 ***             
                                  (0.02)                
mother_eduy                       -0.05 ***    -0.05 ***
                                  (0.00)       (0.00)   
child_age                                       0.10 ** 
                                               (0.01)   
child_gender                                    0.02    
                                               (0.01)   
--------------------------------------------------------
Num. obs.                       8294         5017       
R^2 (full model)                   0.04         0.10    
R^2 (proj model)                                0.06    
Adj. R^2 (full model)              0.04         0.08    
Adj. R^2 (proj model)                           0.06    
Num. groups: wav                                3       
Num. groups: mother_birth_y   

NOTE: 2,008 observations removed because of NA values (LHS: 280, RHS: 1,784).
NOTE: 5,285 observations removed because of NA values (LHS: 280, RHS: 1,784, Fixed-effects: 5,020).
